##### Imports

In [1]:
import pytest
import os
import pathlib
import pandas as pd
import logging
import matplotlib.pyplot as plt

from tot.dataset import Dataset
from tot.models import BayesianRidgeModel, LinearRegressionModel, NeuralProphetModel, ProphetModel, SeasonalNaiveModel
from tot.experiment import SimpleExperiment, CrossValidationExperiment
from tot.benchmark import SimpleBenchmark, CrossValidationBenchmark
from tot.benchmark import ManualBenchmark, ManualCVBenchmark
from tot.metrics import ERROR_FUNCTIONS

In [2]:
log = logging.getLogger("tot.benchmarking")
log.setLevel("INFO")
log.parent.setLevel("INFO")

In [3]:
# TODO: datasets should have a simple .load() command

DIR = os.path.abspath('..')
DATA_DIR = os.path.join(DIR, "tests", "test-data")
PEYTON_FILE = os.path.join(DATA_DIR, "wp_log_peyton_manning.csv")
AIR_FILE = os.path.join(DATA_DIR, "air_passengers.csv")
ERCOT_FILE = os.path.join(DATA_DIR, "ercot_load.csv")
SAVE_DIR = os.path.join(DIR, "tests", "test-logs")
ERCOT_REGIONS = ["NORTH", "EAST", "FAR_WEST"]
if not os.path.isdir(SAVE_DIR):
    os.makedirs(SAVE_DIR)

EPOCHS = 40
BATCH_SIZE = 64
LR = 1.0
ERCOT_REGIONS = ["NORTH", "EAST", "FAR_WEST"]

PLOT = False

In [4]:
peyton_manning_df = pd.read_csv(PEYTON_FILE)
# air_passengers_df = pd.read_csv(AIR_FILE, nrows=NROWS)
print(peyton_manning_df.shape)
dataset_list = [
    # Dataset(df=air_passengers_df, name="air_passengers", freq="MS", seasonality_mode="multiplicative"),
    Dataset(df=peyton_manning_df, name="peyton_manning", freq="D", seasonalities=[7, 365.25]),
]

(2905, 2)


In [5]:
model_classes_and_params = [
    (NeuralProphetModel, {"n_lags": 10, "n_forecasts": 5, "learning_rate": 0.1, "epochs": EPOCHS}),
    (NeuralProphetModel, {"n_lags": 5, "n_forecasts": 5, "learning_rate": 0.1, "epochs": EPOCHS}),
    (NeuralProphetModel, {"n_lags": 0,"n_forecasts": 1}),
    (ProphetModel, {}),
    (SeasonalNaiveModel, {"n_forecasts": 5, "season_length": 7}),
    (LinearRegressionModel, {"n_lags": 10, "output_chunk_length": 1, "n_forecasts": 5}),
    (BayesianRidgeModel, {"n_lags": 10, "output_chunk_length": 1, "n_forecasts": 5}),
]
log.info("{}".format(model_classes_and_params))

In [6]:
benchmark = SimpleBenchmark(
    model_classes_and_params=model_classes_and_params,
    datasets=dataset_list,
    metrics=["MAE", "MSE", "MASE", "RMSE", "MAPE", "SMAPE"],
    test_percentage=25,
)
results_train, results_test = benchmark.run()

log.warning("{}".format(results_test))
# log.warning("{}".format(results_train))


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.761% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.forecaster.__handle_missing_data) - 56 missing dates added.
WARNING - (NP.forecaster.__handle_missing_data) - 56 missing values in column y were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 56 NaN values in column y were auto-imputed.
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.761% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.764% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency -

  0%|          | 0/133 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.37E-01, min: 1.69E+00


  0%|          | 0/133 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.37E-01, min: 3.00E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.23E-01
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.761% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 98.761% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.449% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.449% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
15:58:07 - cmdstanpy - INFO - Chain [1] start processing
15:58:07 - cmdstanpy -

...

  0%|          | 0/727 [00:00<?, ?it/s]

59 missing values in column y were detected in total. 


  0%|          | 0/2209 [00:00<?, ?it/s]

  0%|          | 0/727 [00:00<?, ?it/s]

             data                  model  \
0  peyton_manning          NeuralProphet   
1  peyton_manning          NeuralProphet   
2  peyton_manning          NeuralProphet   
3  peyton_manning                Prophet   
4  peyton_manning          SeasonalNaive   
5  peyton_manning  LinearRegressionModel   
6  peyton_manning          BayesianRidge   

                                              params  \
0  {'n_lags': 10, 'n_forecasts': 5, 'learning_rat...   
1  {'n_lags': 5, 'n_forecasts': 5, 'learning_rate...   
2  {'n_lags': 0, 'n_forecasts': 1, '_data_params'...   
3  {'_data_params': {'seasonalities': [7, 365.25]...   
4  {'n_forecasts': 5, 'season_length': 7, '_data_...   
5  {'n_lags': 10, 'output_chunk_length': 1, 'n_fo...   
6  {'n_lags': 10, 'output_chunk_length': 1, 'n_fo...   

                                          experiment       MAE       MSE  \
0  peyton_manning_NeuralProphet_n_lags_10_n_forec...  0.295409  0.205498   
1  peyton_manning_NeuralProphet_n_lags_5_n_for